In [1]:
import pandas as pd
# import lux
import numpy as np
from ludwig.api import LudwigModel

/home/lux_t1/miniconda3/envs/ludwig/lib/python3.8/site-packages/typeguard/__init__.py:917: UserWarning: no type annotations present -- not typechecking tensorflow_addons.layers.max_unpooling_2d.MaxUnpooling2D.__init__
  warn('no type annotations present -- not typechecking {}'.format(function_name(func)))


In [27]:
df = pd.read_excel("data/features.xlsx")

In [28]:
df.set_index("Auftragsnummer", inplace = True)

In [29]:
df.drop("Untersucher 2", axis = 1, inplace = True)

In [30]:
df.dropna(axis = 0, how ="any", inplace = True)

In [6]:
# cols = df.columns[4:].to_list()
# drop_cols = []
# for col in cols:
#     if df[col].sum()<100:
#         drop_cols.append(col)
# drop_cols

In [77]:
# df.drop(drop_cols, axis = 1, inplace = True)

In [32]:
df.drop("Unnamed: 187", axis = 1, inplace = True)

In [34]:
df  = df.rename(columns = {_: str(_) for _ in df.columns})

In [18]:
# for col in df.columns:
#     print(col)

In [ ]:
# {input_features: [{name: doc_text, type: text, encoder: rnn}], output_features: [{name: class, type: category}], training: {epochs: 50}}

In [35]:
config = {}

config["input_features"] = [{"name": "U-Dauer", "type": "numerical"},{"name": "Fachgebiet", "type": "category"}]

config["output_features"] = [{"name": _, "type": "numerical"} for _ in df.columns[3:]]


In [36]:
model = LudwigModel(config)

In [37]:
train_stats, _, _  = model.train(dataset=df)

In [38]:
model.save("ludwig_default_model")

In [42]:
evaluation_statistics, predictions, output_directory = model.evaluate(
  dataset=df,
  split='full',
  batch_size=128,
  skip_save_unprocessed_output=True,
  skip_save_predictions=True,
  skip_save_eval_stats=True,
  collect_predictions=False,
  collect_overall_stats=False,
  output_directory='results',
  debug=False
)

In [43]:
evaluation_statistics

{'11126032': {'loss': 0.034893777,
  'error': 0.0028807437,
  'mean_squared_error': 0.034893785,
  'mean_absolute_error': 0.05780947,
  'r2': 0.20472878},
 '11125327': {'loss': 0.035567608,
  'error': 0.0030784488,
  'mean_squared_error': 0.035567608,
  'mean_absolute_error': 0.071541384,
  'r2': 0.3119067},
 '10168546': {'loss': 0.0107206125,
  'error': -0.00408371,
  'mean_squared_error': 0.010720615,
  'mean_absolute_error': 0.02628078,
  'r2': 0.054282725},
 '11129570': {'loss': 0.14528306,
  'error': -0.007384086,
  'mean_squared_error': 0.14528307,
  'mean_absolute_error': 0.3015355,
  'r2': 0.041095555},
 '10132597': {'loss': 0.027035842,
  'error': -0.0034402867,
  'mean_squared_error': 0.02703584,
  'mean_absolute_error': 0.09420267,
  'r2': -0.47131813},
 '11101690': {'loss': 0.018383004,
  'error': 0.006693897,
  'mean_squared_error': 0.018383004,
  'mean_absolute_error': 0.04423982,
  'r2': 0.056705892},
 '10196004': {'loss': 0.038946167,
  'error': -0.003973555,
  'mean_sq

In [44]:
df.head()

,U-Dauer,Fachgebiet,10129221,11126032,11125327,10168546,11129570,10132597,11101690,10196004,...,10174080,10172946,11100018,10191021,10148636,10119146,10178772,11122372,10164515,10157942
Auftragsnummer,,,,,,,,,,,,,,,,,,,,,
100006112,24.0,Endosonographie,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100002310,16.0,Gastroskopie,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97671673,25.0,Koloskopie,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98573033,10.0,Gastroskopie,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100011189,14.0,Gastroskopie,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [45]:
to_predict = pd.DataFrame({"U-Dauer": [15.0, 25.0], "Fachgebiet": ["Gastroskopie", "Koloskopie"]})
to_predict

,U-Dauer,Fachgebiet
0,15.0,Gastroskopie
1,25.0,Koloskopie


In [46]:
predictions, output_directory = model.predict(dataset = to_predict, output_directory = "test_predict")

In [47]:
predictions

,11126032_predictions,11125327_predictions,10168546_predictions,11129570_predictions,10132597_predictions,11101690_predictions,10196004_predictions,10186917_predictions,11101853_predictions,10125432_predictions,...,10174080_predictions,10172946_predictions,11100018_predictions,10191021_predictions,10148636_predictions,10119146_predictions,10178772_predictions,11122372_predictions,10164515_predictions,10157942_predictions
0,0.011632,0.011317,0.008327,0.211584,0.073041,0.010038,-0.021195,-0.020532,0.003784,0.012817,...,-0.029191,0.002050,-0.003363,0.036079,0.001789,-0.028626,-0.002522,0.007832,-0.003414,-0.007598
1,0.003942,0.038562,0.006606,0.265490,-0.100256,0.023079,0.156066,-0.004989,-0.000099,0.015911,...,0.026499,0.045396,-0.029334,-0.146961,0.000820,0.116118,-0.000833,0.008573,-0.017771,-0.015485


In [82]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split

In [83]:
seed = 42

In [84]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=seed)

In [85]:
model = MultiOutputRegressor(GradientBoostingRegressor(random_state=seed)).fit(X_train, y_train)

In [86]:
model.score(X_test, y_test)

0.2504132462601212

In [88]:
X.shape

(1753, 13)

In [ ]:
"""
'ERCP', 
'Endosonographie', 
'Enteroskopie/Single-,Doppelballon',
'Gastroskopie', 
'Ileokoloskopie', 
'Kapselendoskopie', 
'Koloskopie',
'MC_Gastroskopie', 
'MC_Ileokoloskopie', 
'MC_Koloskopie', 
'MENELMGKE',
'Rektosigmoidoskopie'
""" 

In [90]:
to_predict = np.array([[0,0,0,0,0,0,1,0,0,0,0,0,24]])

In [91]:
model.predict(to_predict)

array([[0.21291703, 0.00091866, 0.14777661, 0.24903106]])